# Esempio 5: Selezione Progetti con Budget Limitato

## Ricerca Operativa - Problema dello Zaino (Knapsack Problem)

**Obiettivo**: Imparare a risolvere problemi di selezione con vincoli multipli usando programmazione binaria.

---

## 📋 Descrizione del Problema

Un'azienda deve **selezionare quali progetti di innovazione finanziare** con risorse limitate.

### 🎯 Il Problema dello Zaino (Knapsack)

**Analogia classica:**  
Avete uno zaino con capacità limitata e dovete scegliere quali oggetti portare per massimizzare il valore totale.

**Nel nostro caso:**
- **Zaino** = Budget e risorse disponibili
- **Oggetti** = Progetti da valutare
- **Peso** = Costi e risorse necessarie
- **Valore** = VAN (Valore Attuale Netto)

### Dati disponibili:

| Progetto | VAN (k€) | Investimento (k€) | Risorse (FTE) | Durata (mesi) |
|----------|----------|-------------------|---------------|---------------|
| P1       | 250      | 120               | 3             | 12            |
| P2       | 180      | 80                | 2             | 8             |
| P3       | 320      | 150               | 4             | 15            |
| P4       | 200      | 100               | 2             | 10            |
| P5       | 140      | 70                | 2             | 6             |
| P6       | 280      | 130               | 3             | 14            |

**VAN (Valore Attuale Netto)**: Misura il valore creato da un progetto in termini monetari, scontato al presente.

### Vincoli:
- **Budget totale disponibile**: 300 k€
- **Risorse umane disponibili**: 7 FTE (Full-Time Equivalent)
- I progetti sono **indivisibili** (tutto o niente)

### Obiettivo:
Selezionare il **portafoglio di progetti che massimizza il VAN totale** rispettando i vincoli di budget e risorse.

### 🤔 Domande strategiche:
1. Dobbiamo fare il progetto con VAN più alto (P3)?
2. È meglio fare pochi progetti grandi o molti piccoli?
3. Come bilanciare budget e risorse umane?

## 📐 Modello Matematico

### Insiemi:
- $P = \{1, 2, 3, 4, 5, 6\}$ = insieme dei progetti

### Parametri:
- $v_i$ = VAN del progetto $i$ (k€)
- $c_i$ = costo (investimento) del progetto $i$ (k€)
- $r_i$ = risorse necessarie per il progetto $i$ (FTE)
- $B$ = budget totale disponibile (300 k€)
- $R$ = risorse totali disponibili (7 FTE)

### Variabili decisionali:

**Variabili binarie:**
$$x_i \in \{0, 1\} = \begin{cases}
1 & \text{se selezioniamo il progetto } i \\
0 & \text{se NON selezioniamo il progetto } i
\end{cases}$$

### Funzione obiettivo:
$$\max z = \sum_{i \in P} v_i \cdot x_i$$

Massimizziamo la somma dei VAN dei progetti selezionati.

### Vincoli:

**1. Vincolo di budget:**
$$\sum_{i \in P} c_i \cdot x_i \leq B$$
La somma dei costi dei progetti selezionati non può superare il budget.

**2. Vincolo di risorse:**
$$\sum_{i \in P} r_i \cdot x_i \leq R$$
La somma delle risorse necessarie non può superare le risorse disponibili.

**3. Vincoli di binarietà:**
$$x_i \in \{0, 1\} \quad \forall i \in P$$

### Tipo di problema:
- **Binary Integer Programming (BIP)** - Solo variabili binarie
- **Multidimensional Knapsack Problem** - Due vincoli di capacità
- **NP-hard** - Computazionalmente difficile per grandi istanze

## 💻 Implementazione in Gurobi

### Step 1: Import e Setup

In [1]:
# Import delle librerie necessarie
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations

%matplotlib inline

# Stile grafici
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("Librerie importate con successo!")
print(f"Versione Gurobi: {gp.gurobi.version()}")

Librerie importate con successo!
Versione Gurobi: (12, 0, 3)


### Step 2: Definizione dei Dati del Problema

In [2]:
# Insieme dei progetti
progetti = [1, 2, 3, 4, 5, 6]

# VAN (Valore Attuale Netto) per progetto (k€)
van = {
    1: 250,
    2: 180,
    3: 320,
    4: 200,
    5: 140,
    6: 280
}

# Costo (investimento) per progetto (k€)
costo = {
    1: 120,
    2: 80,
    3: 150,
    4: 100,
    5: 70,
    6: 130
}

# Risorse necessarie (FTE - Full Time Equivalent)
risorse = {
    1: 3,
    2: 2,
    3: 4,
    4: 2,
    5: 2,
    6: 3
}

# Durata progetti (mesi) - informazione aggiuntiva
durata = {
    1: 12,
    2: 8,
    3: 15,
    4: 10,
    5: 6,
    6: 14
}

# Vincoli di capacità
budget_disponibile = 300  # k€
risorse_disponibili = 7   # FTE

print("\n=== DATI DEL PROBLEMA ===")
print(f"\nNumero progetti candidati: {len(progetti)}")
print(f"Budget disponibile: {budget_disponibile} k€")
print(f"Risorse disponibili: {risorse_disponibili} FTE")
print(f"\nVAN totale se facessimo tutti i progetti: {sum(van.values())} k€")
print(f"Costo totale se facessimo tutti i progetti: {sum(costo.values())} k€")
print(f"Risorse totali necessarie: {sum(risorse.values())} FTE")
print(f"\n⚠️ Non possiamo fare tutti i progetti! Dobbiamo scegliere.")


=== DATI DEL PROBLEMA ===

Numero progetti candidati: 6
Budget disponibile: 300 k€
Risorse disponibili: 7 FTE

VAN totale se facessimo tutti i progetti: 1370 k€
Costo totale se facessimo tutti i progetti: 650 k€
Risorse totali necessarie: 16 FTE

⚠️ Non possiamo fare tutti i progetti! Dobbiamo scegliere.


In [3]:
# Visualizzazione tabellare completa
df_progetti = pd.DataFrame({
    'Progetto': [f'P{i}' for i in progetti],
    'VAN (k€)': [van[i] for i in progetti],
    'Costo (k€)': [costo[i] for i in progetti],
    'Risorse (FTE)': [risorse[i] for i in progetti],
    'Durata (mesi)': [durata[i] for i in progetti],
    'ROI (VAN/Costo)': [van[i]/costo[i] for i in progetti],
    'VAN/FTE': [van[i]/risorse[i] for i in progetti]
})

# Ordina per VAN decrescente
df_progetti_sorted = df_progetti.sort_values('VAN (k€)', ascending=False)

print("\n=== TABELLA PROGETTI (ordinati per VAN) ===")
print(df_progetti_sorted.to_string(index=False))

print("\n💡 METRICHE DI EFFICIENZA:")
print("  ROI = VAN / Costo (quanto valore per ogni € investito)")
print("  VAN/FTE = quanto valore per ogni risorsa umana utilizzata")


=== TABELLA PROGETTI (ordinati per VAN) ===
Progetto  VAN (k€)  Costo (k€)  Risorse (FTE)  Durata (mesi)  ROI (VAN/Costo)    VAN/FTE
      P3       320         150              4             15         2.133333  80.000000
      P6       280         130              3             14         2.153846  93.333333
      P1       250         120              3             12         2.083333  83.333333
      P4       200         100              2             10         2.000000 100.000000
      P2       180          80              2              8         2.250000  90.000000
      P5       140          70              2              6         2.000000  70.000000

💡 METRICHE DI EFFICIENZA:
  ROI = VAN / Costo (quanto valore per ogni € investito)
  VAN/FTE = quanto valore per ogni risorsa umana utilizzata


### Step 3: Analisi Preliminare

Prima di risolvere il problema, esploriamo alcune soluzioni "intuitive".

In [4]:
print("\n" + "="*70)
print("              ANALISI SOLUZIONI INTUITIVE (GREEDY)")
print("="*70)

# Soluzione 1: Greedy per VAN (prendi progetti con VAN più alto)
print("\n📊 STRATEGIA 1: Greedy per VAN massimo")
print("-" * 70)
progetti_ordinati_van = sorted(progetti, key=lambda i: van[i], reverse=True)
print(f"Ordine progetti per VAN: {['P'+str(i) for i in progetti_ordinati_van]}")

selezionati_van = []
budget_usato = 0
risorse_usate = 0

for p in progetti_ordinati_van:
    if budget_usato + costo[p] <= budget_disponibile and risorse_usate + risorse[p] <= risorse_disponibili:
        selezionati_van.append(p)
        budget_usato += costo[p]
        risorse_usate += risorse[p]

van_totale_1 = sum(van[p] for p in selezionati_van)
print(f"\nProgetti selezionati: {['P'+str(p) for p in selezionati_van]}")
print(f"VAN totale: {van_totale_1} k€")
print(f"Budget utilizzato: {budget_usato}/{budget_disponibile} k€")
print(f"Risorse utilizzate: {risorse_usate}/{risorse_disponibili} FTE")

# Soluzione 2: Greedy per ROI (VAN/Costo)
print("\n📊 STRATEGIA 2: Greedy per ROI (VAN/Costo)")
print("-" * 70)
progetti_ordinati_roi = sorted(progetti, key=lambda i: van[i]/costo[i], reverse=True)
print(f"Ordine progetti per ROI: {['P'+str(i) for i in progetti_ordinati_roi]}")

selezionati_roi = []
budget_usato = 0
risorse_usate = 0

for p in progetti_ordinati_roi:
    if budget_usato + costo[p] <= budget_disponibile and risorse_usate + risorse[p] <= risorse_disponibili:
        selezionati_roi.append(p)
        budget_usato += costo[p]
        risorse_usate += risorse[p]

van_totale_2 = sum(van[p] for p in selezionati_roi)
print(f"\nProgetti selezionati: {['P'+str(p) for p in selezionati_roi]}")
print(f"VAN totale: {van_totale_2} k€")
print(f"Budget utilizzato: {sum(costo[p] for p in selezionati_roi)}/{budget_disponibile} k€")
print(f"Risorse utilizzate: {sum(risorse[p] for p in selezionati_roi)}/{risorse_disponibili} FTE")

# Soluzione 3: Greedy per VAN/FTE
print("\n📊 STRATEGIA 3: Greedy per efficienza risorse (VAN/FTE)")
print("-" * 70)
progetti_ordinati_fte = sorted(progetti, key=lambda i: van[i]/risorse[i], reverse=True)
print(f"Ordine progetti per VAN/FTE: {['P'+str(i) for i in progetti_ordinati_fte]}")

selezionati_fte = []
budget_usato = 0
risorse_usate = 0

for p in progetti_ordinati_fte:
    if budget_usato + costo[p] <= budget_disponibile and risorse_usate + risorse[p] <= risorse_disponibili:
        selezionati_fte.append(p)
        budget_usato += costo[p]
        risorse_usate += risorse[p]

van_totale_3 = sum(van[p] for p in selezionati_fte)
print(f"\nProgetti selezionati: {['P'+str(p) for p in selezionati_fte]}")
print(f"VAN totale: {van_totale_3} k€")
print(f"Budget utilizzato: {sum(costo[p] for p in selezionati_fte)}/{budget_disponibile} k€")
print(f"Risorse utilizzate: {sum(risorse[p] for p in selezionati_fte)}/{risorse_disponibili} FTE")

print("\n" + "="*70)
print("\n🤔 Quale strategia è migliore? Vediamo la soluzione OTTIMA...")
print("="*70)


              ANALISI SOLUZIONI INTUITIVE (GREEDY)

📊 STRATEGIA 1: Greedy per VAN massimo
----------------------------------------------------------------------
Ordine progetti per VAN: ['P3', 'P6', 'P1', 'P4', 'P2', 'P5']

Progetti selezionati: ['P3', 'P6']
VAN totale: 600 k€
Budget utilizzato: 280/300 k€
Risorse utilizzate: 7/7 FTE

📊 STRATEGIA 2: Greedy per ROI (VAN/Costo)
----------------------------------------------------------------------
Ordine progetti per ROI: ['P2', 'P6', 'P3', 'P1', 'P4', 'P5']

Progetti selezionati: ['P2', 'P6', 'P5']
VAN totale: 600 k€
Budget utilizzato: 280/300 k€
Risorse utilizzate: 7/7 FTE

📊 STRATEGIA 3: Greedy per efficienza risorse (VAN/FTE)
----------------------------------------------------------------------
Ordine progetti per VAN/FTE: ['P4', 'P6', 'P2', 'P1', 'P3', 'P5']

Progetti selezionati: ['P4', 'P6', 'P5']
VAN totale: 620 k€
Budget utilizzato: 300/300 k€
Risorse utilizzate: 7/7 FTE


🤔 Quale strategia è migliore? Vediamo la soluzione OTT

### Step 4: Creazione del Modello Gurobi

In [5]:
# Creiamo il modello
modello = gp.Model("Selezione_Progetti_Knapsack")

# Opzionale: output silenzioso
modello.Params.OutputFlag = 0

print("Modello BIP (Binary Integer Programming) creato con successo!")

Set parameter Username
Set parameter LicenseID to value 2704465
Academic license - for non-commercial use only - expires 2026-09-08
Modello BIP (Binary Integer Programming) creato con successo!


### Step 5: Definizione delle Variabili Decisionali

Solo **variabili binarie** in questo problema!

In [6]:
# Variabili binarie: x[i] = 1 se selezioniamo il progetto i, 0 altrimenti
x = {}
for i in progetti:
    x[i] = modello.addVar(
        vtype=GRB.BINARY,  # ← Variabile binaria (0 o 1)
        name=f"x_P{i}"
    )

modello.update()

print("\n=== VARIABILI DECISIONALI ===")
print(f"Variabili binarie: {len(progetti)}")
print("\nOgni variabile x_i può essere:")
print("  x_i = 1 → Selezioniamo il progetto i")
print("  x_i = 0 → NON selezioniamo il progetto i")
print(f"\nCombinazioni possibili: 2^{len(progetti)} = {2**len(progetti)} soluzioni")
print("(Ma non tutte sono ammissibili - molte violano i vincoli!)")


=== VARIABILI DECISIONALI ===
Variabili binarie: 6

Ogni variabile x_i può essere:
  x_i = 1 → Selezioniamo il progetto i
  x_i = 0 → NON selezioniamo il progetto i

Combinazioni possibili: 2^6 = 64 soluzioni
(Ma non tutte sono ammissibili - molte violano i vincoli!)


### Step 6: Definizione della Funzione Obiettivo

In [7]:
# Funzione obiettivo: massimizzare il VAN totale
funzione_obiettivo = gp.quicksum(van[i] * x[i] for i in progetti)

modello.setObjective(funzione_obiettivo, GRB.MAXIMIZE)

print("\n=== FUNZIONE OBIETTIVO ===")
print("Massimizza: Σ VAN_i * x_i")
print("\nCioè: massimizzare la somma dei VAN dei progetti selezionati")
print("\nEsempio: Se selezioniamo P1, P3 e P5:")
print(f"  VAN totale = {van[1]} + {van[3]} + {van[5]} = {van[1] + van[3] + van[5]} k€")


=== FUNZIONE OBIETTIVO ===
Massimizza: Σ VAN_i * x_i

Cioè: massimizzare la somma dei VAN dei progetti selezionati

Esempio: Se selezioniamo P1, P3 e P5:
  VAN totale = 250 + 320 + 140 = 710 k€


### Step 7: Aggiunta dei Vincoli

In [8]:
# VINCOLO 1: Budget
# La somma dei costi dei progetti selezionati non può superare il budget
vincolo_budget = modello.addConstr(
    gp.quicksum(costo[i] * x[i] for i in progetti) <= budget_disponibile,
    name="Vincolo_Budget"
)

print("\n=== VINCOLO DI BUDGET ===")
print(f"Σ Costo_i * x_i ≤ {budget_disponibile} k€")
print("\nEsempio: Se selezioniamo P1, P3 e P5:")
print(f"  Costo totale = {costo[1]} + {costo[3]} + {costo[5]} = {costo[1] + costo[3] + costo[5]} k€")
print(f"  {'✓ OK' if costo[1] + costo[3] + costo[5] <= budget_disponibile else '✗ Viola il vincolo'}")


=== VINCOLO DI BUDGET ===
Σ Costo_i * x_i ≤ 300 k€

Esempio: Se selezioniamo P1, P3 e P5:
  Costo totale = 120 + 150 + 70 = 340 k€
  ✗ Viola il vincolo


In [9]:
# VINCOLO 2: Risorse umane
# La somma delle risorse necessarie non può superare le risorse disponibili
vincolo_risorse = modello.addConstr(
    gp.quicksum(risorse[i] * x[i] for i in progetti) <= risorse_disponibili,
    name="Vincolo_Risorse"
)

modello.update()

print("\n=== VINCOLO DI RISORSE ===")
print(f"Σ Risorse_i * x_i ≤ {risorse_disponibili} FTE")
print("\nEsempio: Se selezioniamo P1, P3 e P5:")
print(f"  Risorse totali = {risorse[1]} + {risorse[3]} + {risorse[5]} = {risorse[1] + risorse[3] + risorse[5]} FTE")
print(f"  {'✓ OK' if risorse[1] + risorse[3] + risorse[5] <= risorse_disponibili else '✗ Viola il vincolo'}")


=== VINCOLO DI RISORSE ===
Σ Risorse_i * x_i ≤ 7 FTE

Esempio: Se selezioniamo P1, P3 e P5:
  Risorse totali = 3 + 4 + 2 = 9 FTE
  ✗ Viola il vincolo


### Step 8: Riepilogo del Modello

In [10]:
print("\n" + "="*70)
print("                    RIEPILOGO MODELLO BIP")
print("="*70)
print(f"\n📊 VARIABILI:")
print(f"  Totali: {modello.NumVars}")
print(f"  Tipo: Tutte BINARIE (0 o 1)")

print(f"\n🔗 VINCOLI:")
print(f"  Totali: {modello.NumConstrs}")
print(f"  - Vincolo di budget: 1")
print(f"  - Vincolo di risorse: 1")

print(f"\n🎯 TIPO DI MODELLO: Binary Integer Programming (BIP)")
print(f"  Sottocategoria: Multidimensional Knapsack Problem")
print(f"  Complessità: NP-hard")

print(f"\n🔢 SPAZIO DI RICERCA:")
print(f"  Soluzioni possibili: {2**len(progetti)}")
print(f"  Soluzioni ammissibili: < {2**len(progetti)} (molte violano vincoli)")
print("\n" + "="*70)


                    RIEPILOGO MODELLO BIP

📊 VARIABILI:
  Totali: 6
  Tipo: Tutte BINARIE (0 o 1)

🔗 VINCOLI:
  Totali: 2
  - Vincolo di budget: 1
  - Vincolo di risorse: 1

🎯 TIPO DI MODELLO: Binary Integer Programming (BIP)
  Sottocategoria: Multidimensional Knapsack Problem
  Complessità: NP-hard

🔢 SPAZIO DI RICERCA:
  Soluzioni possibili: 64
  Soluzioni ammissibili: < 64 (molte violano vincoli)



## 🚀 Risoluzione del Modello

In [11]:
# Risolviamo il modello
print("\n=== RISOLUZIONE IN CORSO ===")
print("Sto cercando la soluzione ottima...\n")

modello.optimize()

# Controlliamo lo stato della soluzione
if modello.status == GRB.OPTIMAL:
    print("✅ SOLUZIONE OTTIMA TROVATA!")
    print(f"\nTempo di risoluzione: {modello.Runtime:.4f} secondi")
    print(f"Gap di ottimalità: {modello.MIPGap * 100:.6f}%")
    print(f"Nodi esplorati: {modello.NodeCount}")
elif modello.status == GRB.INFEASIBLE:
    print("❌ Il modello è INAMMISSIBILE")
elif modello.status == GRB.UNBOUNDED:
    print("⚠️ Il modello è ILLIMITATO")
else:
    print(f"⚠️ Status: {modello.status}")


=== RISOLUZIONE IN CORSO ===
Sto cercando la soluzione ottima...

✅ SOLUZIONE OTTIMA TROVATA!

Tempo di risoluzione: 0.0110 secondi
Gap di ottimalità: 0.000000%
Nodi esplorati: 1.0


## 📊 Risultati della Soluzione Ottima

In [12]:
if modello.status == GRB.OPTIMAL:
    # Salviamo la soluzione
    soluzione = {i: x[i].X for i in progetti}
    progetti_selezionati = [i for i in progetti if soluzione[i] > 0.5]
    progetti_non_selezionati = [i for i in progetti if soluzione[i] < 0.5]
    
    # Metriche soluzione ottima
    van_totale_ottimo = modello.ObjVal
    budget_utilizzato = sum(costo[i] for i in progetti_selezionati)
    risorse_utilizzate = sum(risorse[i] for i in progetti_selezionati)
    
    print("\n" + "="*70)
    print("                  PORTAFOGLIO PROGETTI OTTIMALE")
    print("="*70)
    
    print(f"\n💰 VAN TOTALE MASSIMO: {van_totale_ottimo:.0f} k€")
    print("="*70)
    
    print("\n✅ PROGETTI SELEZIONATI:")
    print("-" * 70)
    for i in progetti_selezionati:
        print(f"  P{i}: VAN={van[i]:>3} k€, Costo={costo[i]:>3} k€, Risorse={risorse[i]} FTE, Durata={durata[i]:>2} mesi")
    
    print("\n❌ PROGETTI NON SELEZIONATI:")
    print("-" * 70)
    for i in progetti_non_selezionati:
        print(f"  P{i}: VAN={van[i]:>3} k€, Costo={costo[i]:>3} k€, Risorse={risorse[i]} FTE, Durata={durata[i]:>2} mesi")
    
    print("\n📊 UTILIZZO RISORSE:")
    print("="*70)
    print(f"Budget:   {budget_utilizzato:>3}/{budget_disponibile} k€ ({budget_utilizzato/budget_disponibile*100:.1f}%)")
    print(f"Risorse:  {risorse_utilizzate:>3}/{risorse_disponibili} FTE ({risorse_utilizzate/risorse_disponibili*100:.1f}%)")
    
    budget_residuo = budget_disponibile - budget_utilizzato
    risorse_residue = risorse_disponibili - risorse_utilizzate
    print(f"\nCapacità residua:")
    print(f"  Budget non utilizzato: {budget_residuo} k€")
    print(f"  Risorse non utilizzate: {risorse_residue} FTE")
    
    print("\n" + "="*70)


                  PORTAFOGLIO PROGETTI OTTIMALE

💰 VAN TOTALE MASSIMO: 630 k€

✅ PROGETTI SELEZIONATI:
----------------------------------------------------------------------
  P1: VAN=250 k€, Costo=120 k€, Risorse=3 FTE, Durata=12 mesi
  P2: VAN=180 k€, Costo= 80 k€, Risorse=2 FTE, Durata= 8 mesi
  P4: VAN=200 k€, Costo=100 k€, Risorse=2 FTE, Durata=10 mesi

❌ PROGETTI NON SELEZIONATI:
----------------------------------------------------------------------
  P3: VAN=320 k€, Costo=150 k€, Risorse=4 FTE, Durata=15 mesi
  P5: VAN=140 k€, Costo= 70 k€, Risorse=2 FTE, Durata= 6 mesi
  P6: VAN=280 k€, Costo=130 k€, Risorse=3 FTE, Durata=14 mesi

📊 UTILIZZO RISORSE:
Budget:   300/300 k€ (100.0%)
Risorse:    7/7 FTE (100.0%)

Capacità residua:
  Budget non utilizzato: 0 k€
  Risorse non utilizzate: 0 FTE



In [13]:
if modello.status == GRB.OPTIMAL:
    print("\n" + "="*70)
    print("         CONFRONTO: SOLUZIONE OTTIMA vs SOLUZIONI GREEDY")
    print("="*70)
    
    risultati_confronto = pd.DataFrame({
        'Strategia': ['Greedy VAN', 'Greedy ROI', 'Greedy VAN/FTE', 'OTTIMA (Gurobi)'],
        'Progetti': [
            ', '.join([f'P{p}' for p in selezionati_van]),
            ', '.join([f'P{p}' for p in selezionati_roi]),
            ', '.join([f'P{p}' for p in selezionati_fte]),
            ', '.join([f'P{p}' for p in progetti_selezionati])
        ],
        'VAN Totale (k€)': [van_totale_1, van_totale_2, van_totale_3, van_totale_ottimo],
        'Gap vs Ottimo': [
            f"{van_totale_ottimo - van_totale_1:.0f} k€",
            f"{van_totale_ottimo - van_totale_2:.0f} k€",
            f"{van_totale_ottimo - van_totale_3:.0f} k€",
            "0 k€"
        ],
        '% Ottimalità': [
            f"{van_totale_1/van_totale_ottimo*100:.1f}%",
            f"{van_totale_2/van_totale_ottimo*100:.1f}%",
            f"{van_totale_3/van_totale_ottimo*100:.1f}%",
            "100.0%"
        ]
    })
    
    print("\n" + risultati_confronto.to_string(index=False))
    
    print("\n💡 OSSERVAZIONI:")
    print("-" * 70)
    if van_totale_1 < van_totale_ottimo or van_totale_2 < van_totale_ottimo or van_totale_3 < van_totale_ottimo:
        print("• Le euristiche greedy NON trovano la soluzione ottima!")
        print("• Il solver Gurobi esplora lo spazio delle soluzioni in modo intelligente")
        print("• Per problemi complessi, l'ottimizzazione è essenziale")
    else:
        print("• In questo caso, una delle euristiche greedy ha trovato l'ottimo")
        print("• Ma non è sempre così! Dipende dai dati del problema")
    
    print("="*70)


         CONFRONTO: SOLUZIONE OTTIMA vs SOLUZIONI GREEDY

      Strategia   Progetti  VAN Totale (k€) Gap vs Ottimo % Ottimalità
     Greedy VAN     P3, P6            600.0         30 k€        95.2%
     Greedy ROI P2, P6, P5            600.0         30 k€        95.2%
 Greedy VAN/FTE P4, P6, P5            620.0         10 k€        98.4%
OTTIMA (Gurobi) P1, P2, P4            630.0          0 k€       100.0%

💡 OSSERVAZIONI:
----------------------------------------------------------------------
• Le euristiche greedy NON trovano la soluzione ottima!
• Il solver Gurobi esplora lo spazio delle soluzioni in modo intelligente
• Per problemi complessi, l'ottimizzazione è essenziale


## 🧪 Esperimenti: Analisi What-If

### Esperimento 1: Cosa succede con più budget?

In [14]:
# Aumentiamo il budget da 300 a 350 k€
vincolo_budget.RHS = 350

modello.optimize()

if modello.status == GRB.OPTIMAL:
    nuovi_selezionati = [i for i in progetti if x[i].X > 0.5]
    nuovo_van = modello.ObjVal
    
    print("\n" + "="*70)
    print("         SCENARIO: Budget aumentato a 350 k€ (+50 k€)")
    print("="*70)
    print(f"\nVAN originale (budget 300): {van_totale_ottimo:.0f} k€")
    print(f"Nuovo VAN (budget 350): {nuovo_van:.0f} k€")
    print(f"Aumento VAN: {nuovo_van - van_totale_ottimo:.0f} k€")

    
    print(f"\nProgetti originali: {['P'+str(p) for p in progetti_selezionati]}")
    print(f"Progetti con budget maggiore: {['P'+str(p) for p in nuovi_selezionati]}")
    
    nuovi_progetti = set(nuovi_selezionati) - set(progetti_selezionati)
    if nuovi_progetti:
        print(f"\n➕ Progetti aggiunti: {['P'+str(p) for p in nuovi_progetti]}")
    
    print("="*70)

# Ripristino
vincolo_budget.RHS = budget_disponibile
modello.optimize()


         SCENARIO: Budget aumentato a 350 k€ (+50 k€)

VAN originale (budget 300): 630 k€
Nuovo VAN (budget 350): 660 k€
Aumento VAN: 30 k€

Progetti originali: ['P1', 'P2', 'P4']
Progetti con budget maggiore: ['P2', 'P4', 'P6']

➕ Progetti aggiunti: ['P6']


### Esperimento 2: Cosa succede con più risorse umane?

In [15]:
# Aumentiamo le risorse da 7 a 9 FTE
vincolo_risorse.RHS = 9

modello.optimize()

if modello.status == GRB.OPTIMAL:
    nuovi_selezionati = [i for i in progetti if x[i].X > 0.5]
    nuovo_van = modello.ObjVal
    
    print("\n" + "="*70)
    print("         SCENARIO: Risorse aumentate a 9 FTE (+2 FTE)")
    print("="*70)
    print(f"\nVAN originale (7 FTE): {van_totale_ottimo:.0f} k€")
    print(f"Nuovo VAN (9 FTE): {nuovo_van:.0f} k€")
    print(f"Aumento VAN: {nuovo_van - van_totale_ottimo:.0f} k€")
    print(f"Valore per FTE aggiuntivo: {(nuovo_van - van_totale_ottimo)/2:.1f} k€/FTE")
    
    print(f"\nProgetti originali: {['P'+str(p) for p in progetti_selezionati]}")
    print(f"Progetti con più risorse: {['P'+str(p) for p in nuovi_selezionati]}")
    
    print("="*70)

# Ripristino
vincolo_risorse.RHS = risorse_disponibili
modello.optimize()


         SCENARIO: Risorse aumentate a 9 FTE (+2 FTE)

VAN originale (7 FTE): 630 k€
Nuovo VAN (9 FTE): 640 k€
Aumento VAN: 10 k€
Valore per FTE aggiuntivo: 5.0 k€/FTE

Progetti originali: ['P1', 'P2', 'P4']
Progetti con più risorse: ['P2', 'P3', 'P5']


### Esperimento 3: Aggiungere vincoli di dipendenza tra progetti

In [16]:
print("\n" + "="*70)
print("   SCENARIO: P3 può essere fatto SOLO SE facciamo anche P1")
print("="*70)

# Aggiungiamo il vincolo: x_3 <= x_1
# Se x_3 = 1 (facciamo P3), allora x_1 deve essere 1
# Se x_1 = 0 (non facciamo P1), allora x_3 deve essere 0
vincolo_dipendenza = modello.addConstr(
    x[3] <= x[1],
    name="P3_richiede_P1"
)

modello.optimize()

if modello.status == GRB.OPTIMAL:
    nuovi_selezionati = [i for i in progetti if x[i].X > 0.5]
    nuovo_van = modello.ObjVal
    
    print(f"\nVAN originale (senza dipendenze): {van_totale_ottimo:.0f} k€")
    print(f"Nuovo VAN (con dipendenza P3→P1): {nuovo_van:.0f} k€")
    print(f"Differenza: {nuovo_van - van_totale_ottimo:.0f} k€")
    
    print(f"\nProgetti selezionati: {['P'+str(p) for p in nuovi_selezionati]}")
    
    if 3 in nuovi_selezionati:
        if 1 in nuovi_selezionati:
            print("\n✓ Vincolo rispettato: P3 è selezionato E P1 è selezionato")
        else:
            print("\n✗ ERRORE: P3 selezionato ma P1 no!")
    else:
        print("\n✓ P3 non è selezionato (il vincolo non impatta)")
    
    print("\n💡 INTERPRETAZIONE:")
    print("Il vincolo di dipendenza può cambiare la soluzione ottima,")
    print("perché limita le combinazioni ammissibili di progetti.")

# Rimuoviamo il vincolo
modello.remove(vincolo_dipendenza)
modello.optimize()
print("\n" + "="*70)


   SCENARIO: P3 può essere fatto SOLO SE facciamo anche P1

VAN originale (senza dipendenze): 630 k€
Nuovo VAN (con dipendenza P3→P1): 630 k€
Differenza: 0 k€

Progetti selezionati: ['P1', 'P2', 'P4']

✓ P3 non è selezionato (il vincolo non impatta)

💡 INTERPRETAZIONE:
Il vincolo di dipendenza può cambiare la soluzione ottima,
perché limita le combinazioni ammissibili di progetti.



### Esperimento 4: Mutua esclusività tra progetti

In [17]:
print("\n" + "="*70)
print("   SCENARIO: P2 e P4 sono ALTERNATIVI (massimo 1 dei due)")
print("="*70)

# Aggiungiamo il vincolo: x_2 + x_4 <= 1
vincolo_esclusivita = modello.addConstr(
    x[2] + x[4] <= 1,
    name="P2_OR_P4_non_entrambi"
)

modello.optimize()

if modello.status == GRB.OPTIMAL:
    nuovi_selezionati = [i for i in progetti if x[i].X > 0.5]
    nuovo_van = modello.ObjVal
    
    print(f"\nVAN originale: {van_totale_ottimo:.0f} k€")
    print(f"Nuovo VAN (con esclusività P2-P4): {nuovo_van:.0f} k€")
    print(f"Differenza: {nuovo_van - van_totale_ottimo:.0f} k€")
    
    print(f"\nProgetti selezionati: {['P'+str(p) for p in nuovi_selezionati]}")
    
    p2_sel = 2 in nuovi_selezionati
    p4_sel = 4 in nuovi_selezionati
    
    if p2_sel and p4_sel:
        print("\n✗ ERRORE: Sia P2 che P4 sono selezionati!")
    elif p2_sel:
        print("\n✓ Vincolo rispettato: Solo P2 selezionato")
    elif p4_sel:
        print("\n✓ Vincolo rispettato: Solo P4 selezionato")
    else:
        print("\n✓ Nessuno dei due selezionato")
    
    print("\n💡 INTERPRETAZIONE:")
    print("I vincoli di mutua esclusività modellano alternative strategiche")
    print("(es. due tecnologie concorrenti, progetti che usano stessa facility, ecc.)")

# Rimuoviamo il vincolo
modello.remove(vincolo_esclusivita)
modello.optimize()
print("\n" + "="*70)


   SCENARIO: P2 e P4 sono ALTERNATIVI (massimo 1 dei due)

VAN originale: 630 k€
Nuovo VAN (con esclusività P2-P4): 620 k€
Differenza: -10 k€

Progetti selezionati: ['P4', 'P5', 'P6']

✓ Vincolo rispettato: Solo P4 selezionato

💡 INTERPRETAZIONE:
I vincoli di mutua esclusività modellano alternative strategiche
(es. due tecnologie concorrenti, progetti che usano stessa facility, ecc.)



### Esperimento 5: Numero minimo di progetti

In [18]:
print("\n" + "="*70)
print("   SCENARIO: Dobbiamo fare ALMENO 3 progetti (diversificazione)")
print("="*70)

# Aggiungiamo il vincolo: sum(x_i) >= 3
vincolo_min_progetti = modello.addConstr(
    gp.quicksum(x[i] for i in progetti) >= 3,
    name="Min_3_progetti"
)

modello.optimize()

if modello.status == GRB.OPTIMAL:
    nuovi_selezionati = [i for i in progetti if x[i].X > 0.5]
    nuovo_van = modello.ObjVal
    
    print(f"\nVAN originale: {van_totale_ottimo:.0f} k€ ({len(progetti_selezionati)} progetti)")
    print(f"Nuovo VAN (min 3 progetti): {nuovo_van:.0f} k€ ({len(nuovi_selezionati)} progetti)")
    print(f"Differenza: {nuovo_van - van_totale_ottimo:.0f} k€")
    
    print(f"\nProgetti selezionati: {['P'+str(p) for p in nuovi_selezionati]}")
    
    if len(nuovi_selezionati) >= 3:
        print(f"\n✓ Vincolo rispettato: {len(nuovi_selezionati)} progetti selezionati")
    else:
        print(f"\n✗ ERRORE: Solo {len(nuovi_selezionati)} progetti!")
    
    print("\n💡 INTERPRETAZIONE:")
    if len(progetti_selezionati) < 3:
        print("La soluzione originale aveva meno di 3 progetti.")
        print("Il vincolo ci obbliga a diversificare, riducendo leggermente il VAN.")
    else:
        print("La soluzione originale aveva già ≥3 progetti.")
        print("Il vincolo non cambia la soluzione ottima.")

# Rimuoviamo il vincolo
modello.remove(vincolo_min_progetti)
modello.optimize()
print("\n" + "="*70)


   SCENARIO: Dobbiamo fare ALMENO 3 progetti (diversificazione)

VAN originale: 630 k€ (3 progetti)
Nuovo VAN (min 3 progetti): 630 k€ (3 progetti)
Differenza: 0 k€

Progetti selezionati: ['P1', 'P2', 'P4']

✓ Vincolo rispettato: 3 progetti selezionati

💡 INTERPRETAZIONE:
La soluzione originale aveva già ≥3 progetti.
Il vincolo non cambia la soluzione ottima.



## 🔢 Analisi Enumerativa Completa

Con 6 progetti, possiamo enumerare TUTTE le 64 combinazioni possibili!

In [19]:
print("\n" + "="*70)
print("           ENUMERAZIONE COMPLETA DELLE SOLUZIONI")
print("="*70)

# Generiamo tutte le combinazioni possibili
tutte_soluzioni = []

for r in range(len(progetti) + 1):
    for combo in combinations(progetti, r):
        # Calcola metriche
        van_combo = sum(van[i] for i in combo)
        costo_combo = sum(costo[i] for i in combo)
        risorse_combo = sum(risorse[i] for i in combo)
        
        # Verifica ammissibilità
        ammissibile = (costo_combo <= budget_disponibile and 
                      risorse_combo <= risorse_disponibili)
        
        tutte_soluzioni.append({
            'Progetti': combo,
            'N_progetti': len(combo),
            'VAN': van_combo,
            'Costo': costo_combo,
            'Risorse': risorse_combo,
            'Ammissibile': ammissibile
        })

# Ordina per VAN decrescente
tutte_soluzioni.sort(key=lambda x: x['VAN'], reverse=True)

# Statistiche
num_ammissibili = sum(1 for s in tutte_soluzioni if s['Ammissibile'])
num_non_ammissibili = len(tutte_soluzioni) - num_ammissibili

print(f"\nTotale combinazioni: {len(tutte_soluzioni)}")
print(f"Ammissibili: {num_ammissibili} ({num_ammissibili/len(tutte_soluzioni)*100:.1f}%)")
print(f"Non ammissibili: {num_non_ammissibili} ({num_non_ammissibili/len(tutte_soluzioni)*100:.1f}%)")

print("\n🏆 TOP 10 SOLUZIONI AMMISSIBILI:")
print("-" * 70)
print(f"{'Rank':<5} {'Progetti':<20} {'VAN (k€)':<10} {'Costo':<8} {'Risorse'}")
print("-" * 70)

rank = 1
for sol in tutte_soluzioni:
    if sol['Ammissibile'] and rank <= 10:
        progetti_str = ', '.join([f'P{p}' for p in sol['Progetti']]) if sol['Progetti'] else 'Nessuno'
        marker = '⭐' if tuple(sol['Progetti']) == tuple(progetti_selezionati) else '  '
        print(f"{marker}{rank:<3} {progetti_str:<20} {sol['VAN']:<10} {sol['Costo']:<8} {sol['Risorse']}")
        rank += 1

print("\n⭐ = Soluzione trovata da Gurobi")
print("="*70)


           ENUMERAZIONE COMPLETA DELLE SOLUZIONI

Totale combinazioni: 64
Ammissibili: 28 (43.8%)
Non ammissibili: 36 (56.2%)

🏆 TOP 10 SOLUZIONI AMMISSIBILI:
----------------------------------------------------------------------
Rank  Progetti             VAN (k€)   Costo    Risorse
----------------------------------------------------------------------
⭐1   P1, P2, P4           630        300      7
  2   P4, P5, P6           620        300      7
  3   P3, P6               600        280      7
  4   P2, P5, P6           600        280      7
  5   P1, P4, P5           590        290      7
  6   P1, P3               570        270      7
  7   P1, P2, P5           570        270      7
  8   P1, P6               530        250      6
  9   P3, P4               520        250      6
  10  P2, P4, P5           520        250      6

⭐ = Soluzione trovata da Gurobi


## 📝 Esportazione del Modello

In [20]:
# Esportiamo il modello
modello.write("selezione_progetti_knapsack.lp")
print("\n✅ Modello esportato in 'selezione_progetti_knapsack.lp'")

# Esportiamo la soluzione
modello.write("selezione_progetti_soluzione.sol")
print("✅ Soluzione esportata in 'selezione_progetti_soluzione.sol'")


✅ Modello esportato in 'selezione_progetti_knapsack.lp'
✅ Soluzione esportata in 'selezione_progetti_soluzione.sol'
